In [1]:
import pandas as pd
import scipy as sc
import numpy as np
import re
from matplotlib import pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn import datasets, linear_model
from sklearn.metrics import mean_squared_error, r2_score
from sklearn import tree
from sklearn.metrics import accuracy_score
from sklearn.tree import DecisionTreeClassifier

In [2]:
anime_df = pd.read_csv('anime_df_copy_csv')
genres_df = pd.read_csv('Genre_csv')

anime_df = pd.concat([anime_df, genres_df], axis=1)
anime_df

,anime_name,Format,Number Of Episodes,Episode Duration,status,Popularity,favorites,Studios,Genres,mean_score,...,Mecha,Music,Mystery,Psychological,Romance,Sci_Fi,Slice_of_Life,Sports,Supernatural,Thriller
0,Shingeki no Kyojin,1.0,25.0,24,Finished,522270.0,47674.0,Wit Studio,Action|Drama|Fantasy|Mystery,85.0,...,0,0,1,0,0,0,0,0,0,0
1,DEATH NOTE,1.0,37.0,23,Finished,474846.0,35688.0,MADHOUSE,Mystery|Psychological|Supernatural|Thriller,84.0,...,0,0,1,1,0,0,0,0,1,1
2,Boku no Hero Academia,1.0,13.0,24,Finished,471660.0,23789.0,bones,Action|Adventure|Comedy,79.0,...,0,0,0,0,0,0,0,0,0,0
3,Kimetsu no Yaiba,1.0,26.0,24,Finished,468624.0,40432.0,ufotable,Action|Adventure|Drama|Fantasy|Supernatural,85.0,...,0,0,0,0,0,0,0,0,1,0
4,HUNTER×HUNTER (2011),1.0,148.0,24,Finished,433766.0,56845.0,MADHOUSE,Action|Adventure|Fantasy,90.0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5943,NOMAD: Megalo Box 2 - Hachidori to tabibito.,3.0,1.0,0,Finished,1225.0,11.0,XX,XX,72.0,...,0,0,0,0,0,0,0,0,0,0
5944,The Big O (2003),1.0,13.0,24,Finished,1225.0,54.0,Sunrise,Action|Mecha|Mystery|Psychological|Sci-Fi,73.0,...,1,0,1,1,0,1,0,0,0,0
5945,Zoids Wild,1.0,50.0,25,Finished,1225.0,10.0,OLM,Action|Adventure|Mecha|Sci-Fi,51.0,...,1,0,0,0,0,1,0,0,0,0
5946,Sanctuary,4.0,1.0,67,Finished,1225.0,9.0,Pastel,Action|Drama|Thriller,58.0,...,0,0,0,0,0,0,0,0,0,1


In [3]:
y_binned = anime_df.iloc[:,anime_df.columns.get_loc("mean_score_binned")]

cols=['anime_name','status','Studios','Genres','mean_score','mean_score_binned','Creator']
anime_df.drop(cols, axis=1, inplace=True)

anime_df

,Format,Number Of Episodes,Episode Duration,Popularity,favorites,source,Completed,Current,Planning,Paused,...,Mecha,Music,Mystery,Psychological,Romance,Sci_Fi,Slice_of_Life,Sports,Supernatural,Thriller
0,1.0,25.0,24,522270.0,47674.0,1.0,436995.0,30979.0,36842.0,9191.0,...,0,0,1,0,0,0,0,0,0,0
1,1.0,37.0,23,474846.0,35688.0,1.0,369357.0,26960.0,47004.0,16808.0,...,0,0,1,1,0,0,0,0,1,1
2,1.0,13.0,24,471660.0,23789.0,1.0,388606.0,29172.0,39132.0,7484.0,...,0,0,0,0,0,0,0,0,0,0
3,1.0,26.0,24,468624.0,40432.0,1.0,362290.0,41180.0,48672.0,9883.0,...,0,0,0,0,0,0,0,0,1,0
4,1.0,148.0,24,433766.0,56845.0,1.0,258701.0,65347.0,72050.0,28800.0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5943,3.0,1.0,0,1225.0,11.0,3.0,81.0,73.0,1046.0,16.0,...,0,0,0,0,0,0,0,0,0,0
5944,1.0,13.0,24,1225.0,54.0,3.0,818.0,55.0,326.0,17.0,...,1,0,1,1,0,1,0,0,0,0
5945,1.0,50.0,25,1225.0,10.0,3.0,157.0,135.0,621.0,87.0,...,1,0,0,0,0,1,0,0,0,0
5946,4.0,1.0,67,1225.0,9.0,1.0,503.0,18.0,672.0,16.0,...,0,0,0,0,0,0,0,0,0,1


In [4]:
X_train, X_test, Y_binned_train, Y_binned_test = train_test_split(anime_df, y_binned, test_size=0.3, random_state=42, shuffle=True)

# Create linear regression object
clf = tree.DecisionTreeClassifier(max_depth = 4, min_samples_split = 4)
#clf = tree.DecisionTreeRegressor()

# Train the model using the training sets
clf = clf.fit(X_train, Y_binned_train)

# Make predictions using the testing set
y_binned_pred = clf.predict(X_test)

# 
# print("Accuracy:",metrics.accuracy_score(y_test, y_pred))
# The mean squared error
print("Mean squared error: %.2f" % mean_squared_error(Y_binned_test, y_binned_pred))
# The coefficient of determination: 1 is perfect prediction
print("Coefficient of determination: %.2f" % r2_score(Y_binned_test, y_binned_pred))


# # Model Accuracy, how often is the classifier correct?
# print('Accuracy on training data = %.2f'% metrics.accuracy_score(Y_train, y_pred))

# print('Accuracy on test data = %.2f'% metrics.accuracy_score(y_test, y_pred))

Mean squared error: 0.72
Coefficient of determination: 0.70
